In this module I'm goign to write my first classifier from scratch. If I can do this on my own, it means I understand an important piece of the puzzle. It means I can write a simple classifier from scratch. 

The classifier I'm going to write today is a scrappy version of K-Nearest Neighbors. It's one of the simplest classifiers around. Using the code from the previous module, I'm going to comment out the following lines of code and replace it with code I write myself. The rest of the pipeline will stay exactly the same.
```python
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier()
```

To start, run the original pipeline to see what the accuracy was of the pipeline in the previous module.

# Previous Module Pipeline

In [1]:
from sklearn import datasets
iris = datasets.load_iris()

In [2]:
X = iris.data
y = iris.target

In [3]:
from sklearn.model_selection import train_test_split
X_test, X_train, y_test, y_train = train_test_split(X, y, test_size = .5)

In [4]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier()

In [5]:
classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [6]:
predictions = classifier.predict(X_test)

In [7]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, predictions))

0.96


The accuracy of this module is over 90%, and this is the goal of the classifier I write myself. So in the next section I'm going to comment out the two lines of the classifier and replace it with code for my own classifier. 

Right of the bat, this will break my code, so the first thin I need to do is fix the pipeline. To do this, I'll implement a class for my classifier (which I'll call *ScrappyKNN*). By scrappy, I mean bare bones (i.e. just enough to get it working). 



# Current Module: Get Pipeline Working

to start I'll add some code to the fit() and predict() methods. I'll store the training data in the fit class. You can think of this as just memorizing the data, and you'll see why later on. Inside the predict method, you'll see you need to return a list of predictions. This is because the parameter *X_test* is actually a 2D array, or a list of lists (i.e. each row contains the features for one testing example. To make a prediction for each row, I'll randomly pic a label from the training data and append it to the prediction. At this point the pipeline is working, so lets run it and see how well it does. 


In [8]:
import random

# First implement class for my classifier
class ScrappyKNN():
    # first declare your fit method
    # this method takes the features and labels of the training set as input
    # so I'm adding parameters for these
    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
    
    # as input this receives the features for the testing data
    # and as output this it returns predictions for the labels.
    # The first goal is to get the pipeline working, and understand whta these methods do
    # so before writing the real classifier, let's start with something simpler
    # i'll write a random classifier to start... meaning I'll just guess the label...
    def predict (self, X_test):
        predictions = []
        for row in X_test:
            # this is randomly pulling a value from y_train and appending it to predictions
            label = random.choice(self.y_train)
            predictions.append(label)
        return predictions
    

In [9]:
#  Tasks 

In [10]:
from sklearn import datasets
iris = datasets.load_iris()

In [11]:
X = iris.data
y = iris.target

In [12]:
from sklearn.model_selection import train_test_split
X_test, X_train, y_test, y_train = train_test_split(X, y, test_size = .5)

In [13]:
# Comment out two lines of code from previous module to implement my own classifier

# from sklearn.neighbors import KNeighborsClassifier
# classifier = KNeighborsClassifier()

# now change my pipeline to use it
# looking at the interface for my classifier, there are two I care about...
# fit(): fit does the training
# predict(): predict does the prediction
classifier = ScrappyKNN()

In [14]:
classifier.fit(X_train, y_train)

In [15]:
predictions = classifier.predict(X_test)

In [16]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, predictions))

0.3466666666666667


Recall, there are three different types of flowers in the iris dataset. So the accuracy should be about 33%, which it is. 

I know the interface of a classifier, but when I started this exercise, my accuracy was above 90%. So let's see if I can do better than 33%.

# Intro to k–NN

To do this I'm goning to implment my classifier based on k-nearest neighbors (k-NN). Imagine the dots on the screen are the training data that was memorized in the fit method.

![training data](./assets/dostTrainingData.png)

Now imagine, I'm asked to make a prediction on this testing point that is displayed in gray. How can this be done?
![training data](./assets/dostTrainingDataGray.png)

Well in the nearest neighbor classifier, it works exaclty as it sounds. By finding the training point that's closest to the testing point, I'll find the nearest neighbor.
![training data](./assets/dostTrainingDataGrayKNN.png)

Then I'll predict that the testing point has the same label. For example, in this case I'll guess that the testing dot is *green*, because that's the color of its nearest neighbor.
![training data](./assets/dostTrainingDataGreen.png)

As another example if we had another dot over here...
![training data](./assets/dostTrainingDataGray2.png)

I'd guess that it's red because that's its red.
![training data](./assets/dostTrainingDataRed.png)

Now what about this dot here, right in the middle? Imagine that this dot is equadistant to the nearest green dot and the nearest red dot. There's a tie, so how can I classify it?
![training data](./assets/dostTrainingDataGray3.png)

One option is I can randomly break the tie. But there's another way, and that's where _K_ comes in. _K_ is the number of neighbors to consider when making your prediction.

If _K_ is one, the classifier would only look at the closest neighbor. But if _K_ is 3, the classifier would look at the 3 closest. In this case two of those are green and one is red. To predict, you can vote, and choose the majority class. There is more detail to this algorithm, but that's enough to get started.
![training data](./assets/dostTrainingDataKNN3.png)


# Measure Distance


To code this up, first I'll need a way to find the nearest neighbor. To do this I'll measure the straight line distance between two points, just like you'd do with a rule. There's a formuls for this called the Euclidian distance. 
![training data](./assets/dostEuclideanDistance.png)

It measures the distance between two points, and it works a bit like the Pythagorean Theorem. And the distance to compute is the length of the hypotenuse.
![training data](./assets/dostEuclideanDistance3.png)

Right now I'm computing distance in two dimensional space brecause I only have two features in the dataset.
![training data](./assets/dostDimensions2D.png)

But what if I had three features or 3 dimensions? Well, then I'd be in a cube. I could still visualize how to measure distance in this space with a ruler. 
![training data](./assets/dostDimensions3D.png)

But what if I had four features or 4 dimensions (like I do in iris)? Well now I'm in a hyper cube, and I can't measure this very easy.
![training data](./assets/dostDimensions4D.png)

The good news is the Euclidian distance works the same way regardless of the number of dimensions. With more features, I can just more terms to the equation. 
![training data](./assets/dostEuclideanDistance4.png)

# Let's Code Up the Euclidian Distance

In [17]:
# using a library called scipy to code up Euclidian distance
# scipy is already installed with Anaconda
from scipy.spatial import distance

# here a and b are lists of numeric features
# a is a point from the training data
# b is a point from the testing data
# this function returns the distance between them
def euc(a,b):
    return distance.euclidean(a,b)

This is all the math we need, so now let's take a look at the algoriithm for a classifier.

# Implement Nearest Neighbor Algorithm

To make a prediction for a test point I'll calculate the distance to all the training points. Then I'll predict the testing point has the same label as the closest one.

I'll comment out the random prediction I used in the previous example, and replace it with a method that finds the closest training point to the test point. 

For this example I'll hardcode _K_ equal to 1. So I'm writing a nearest neighbor class. Note: _K_ variable won't appear in the code, since I'm always finding the closest one.

In [18]:
# implementing class for my classifier
class ScrappyKNN():
    # first declare your fit method
    # this method takes the features and labels of the training set as input
    # so I'm adding parameters for these
    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
    
    # as input this receives the features for the testing data
    # and as output this it returns predictions for the labels.
    def predict (self, X_test):
        predictions = []
        for row in X_test:
            # comment out random prediction I made in previous example
            # replace it with a method that finds the closest training point to the test point
            # label = random.choice(self.y_train)
            label = self.closest(row)
            predictions.append(label)
        return predictions
    
    # inside this method I'll loop over all the training points
    # and keep track of the closest one so far. 
    # remember the training was memorized in my training function
    # and X_train contains the features
    def closest(self, row):
        # using variable best_dist to keep track of the shortest distance found so far
        # calculating distance b/w test point and first training point and 
        best_dist = euc(row, self.X_train[0])
        
        # using this variable to keep track of the index of the training point that's closest
        # you'll need thsi later to retrieve its label
        best_index = 0
        
        # let's iterate over all the training points
        for i in range(1, len(self.X_train)):
            dist = euc(row, self.X_train[i])
            
            # every time I find a closer distance, I'll update the variables
            if dist < best_dist:
                best_dist = dist
                
                # use the index to return the label for the closest training example
                best_index = i
        return self.y_train[best_index]
    
    

 At this point, I have have a working nearest neighbor classifier. So I'm going to run it to see what the accuracy is. 

# Run Pipeline

In [19]:
from sklearn.metrics import accuracy_score
from sklearn import datasets
iris = datasets.load_iris()

X = iris.data
y = iris.target

from sklearn.model_selection import train_test_split
X_test, X_train, y_test, y_train = train_test_split(X, y, test_size = .5)

classifier = ScrappyKNN()

classifier.fit(X_train, y_train)
predictions = classifier.predict(X_test)
print(accuracy_score(y_test, predictions))

0.9733333333333334


As you can see, I did it. The accuracy is now over 90%. This value may be different with each run the accuracy may be a bit different due to randomness in the train test split. 

As stated at the very beginning, this is an important milestone. Since I can write this up on my own and understand it, it means I can write a simple classifier from scratch.

# Pros and Cons

### Pros
* Relativiely simple: easy to understand, and works reasonably well with some problems.


### Cons
* Computationally intensive: it's slow because it has to iterate over every training point to make a prediction. 
* Hard to represent relationships between features: some features are more infomative than others, but there's not an easy way to represent that in K-Nearest Neighbors. 


# Coming Up


In the long run I want a classifier that learns mores complex relationships between features, and the label I'm trying to predict. A decision tree is a good example of this and a neural network, like I saw in [tensorflow playground](http://playground.tensorflow.org/#activation=tanh&batchSize=10&dataset=circle&regDataset=reg-plane&learningRate=0.03&regularizationRate=0&noise=0&networkShape=4,2&seed=0.83261&showTestData=false&discretize=false&percTrainData=50&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&hideText=false) is even better.  

# Resource

* [Write your first classifier](https://www.youtube.com/watch?v=AoeEHqVSNOw&list=PLOU2XLYxmsIIuiBfYad6rFYQU_jL2ryal&index=6&t=0s)
* [tensorflow playground](http://playground.tensorflow.org/#activation=tanh&batchSize=10&dataset=circle&regDataset=reg-plane&learningRate=0.03&regularizationRate=0&noise=0&networkShape=4,2&seed=0.83261&showTestData=false&discretize=false&percTrainData=50&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&hideText=false)